In [1]:
import spacy
import pandas as pd
from utilities.utils import *
nlp = spacy.load("de_core_news_sm")
import sys


2023-05-23 16:44:38.454229: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
a = pd.read_csv('data/data_files/transfer_data/thriple_annotator_702.csv')

In [3]:
a

,text,label,pos_string,pos_list
0,Insbesondere die flächendeckende Ausstattung m...,0,ADV DET ADJ NOUN ADP DET ADJ NOUN AUX ADV PUN...,"['ADV', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'A..."
1,Jedoch stand noch ein paar Tage davor nicht fe...,1,ADV VERB ADV DET DET NOUN ADV PART ADV PUNCT ...,"['ADV', 'VERB', 'ADV', 'DET', 'DET', 'NOUN', '..."
2,Immer mehr Menschen scheinen aufzuwachen und w...,0,ADV DET NOUN VERB VERB CCONJ VERB PRON ADP DE...,"['ADV', 'DET', 'NOUN', 'VERB', 'VERB', 'CCONJ'..."
3,"Prof. Jay Bhattacharya, Direktor am Institut f...",1,NOUN PROPN PROPN PUNCT NOUN ADP NOUN ADP PROP...,"['NOUN', 'PROPN', 'PROPN', 'PUNCT', 'NOUN', 'A..."
4,In den großen Städten mussten Kneipen und Loka...,1,ADP DET ADJ NOUN ADJ NOUN CCONJ NOUN VERB PUN...,"['ADP', 'DET', 'ADJ', 'NOUN', 'ADJ', 'NOUN', '..."
...,...,...,...,...
784,In Quarantäne befinden sich zudem der Tagungst...,1,ADP NOUN VERB PRON ADV DET NOUN CCONJ NOUN DE...,"['ADP', 'NOUN', 'VERB', 'PRON', 'ADV', 'DET', ..."
785,Immerhin klopft Merkel dem „kompetenten“ Bunde...,1,ADV VERB PROPN DET PUNCT ADJ PUNCT NOUN PROPN...,"['ADV', 'VERB', 'PROPN', 'DET', 'PUNCT', 'ADJ'..."
786,„Weil sie keine einfache Schanze ist“ und „Feh...,0,PUNCT SCONJ PRON DET ADJ NOUN AUX PUNCT CCONJ...,"['PUNCT', 'SCONJ', 'PRON', 'DET', 'ADJ', 'NOUN..."
787,"„Wir versuchen seit Wochen verzweifelt, irgend...",1,PUNCT PRON VERB ADP NOUN ADV PUNCT ADV ADP DE...,"['PUNCT', 'PRON', 'VERB', 'ADP', 'NOUN', 'ADV'..."


In [5]:
358/(431 + 358) 

0.45373891001267425

In [4]:
a.value_counts('label')

label
1    431
0    358
Name: count, dtype: int64

In [65]:
from back_classes.sentence_classifier import LSTMDataset

In [67]:
test_df = pd.read_csv(
    'test_dataset_samples/sentence_list__link_truth_list__random.csv'
)

In [69]:
split_1,split_2 = split_train_val(test_df)
print(split_1.value_counts('label'))
print(split_2.value_counts('label'))

label
False    22500
True     22500
dtype: int64
label
False    2500
True     2500
dtype: int64


In [68]:
test_df.value_counts('label')

label
False    25000
True     25000
dtype: int64

In [55]:
annotation = pd.read_csv(
    'data_files/FullAnnotated1.csv',sep=';'
)
annotation.head()

,Unnamed: 0,sentence,is_claim,to_exclude
0,0,"Ärger über Gasmasken-Eklat Samstag, 1 Juli, 12...",1,0
1,1,Dabei hatte die Projektleitung eigentlich ein ...,0,0
2,2,Trotz der beiden Bike-Lanes ist der Verkehr au...,1,0
3,3,"Interessant ist, dass es der Civey-Umfrage zuf...",1,0
4,4,Die Salzburger Festspiele aber haben sich zumi...,0,0


In [20]:
len(annotation)

106

In [24]:
annotation = pd.read_csv('data_files/annotated_pos_test.csv')


In [31]:
len(annotation)

97

In [57]:
annotation.to_csv('data_files/annotated_pos_test.csv',index=False)

In [56]:
annotation['text']=annotation['sentence']
annotation['label']=annotation['is_claim']

sentence_list = annotation['text']
pos_list = []
pos_strings = []
for sentence in sentence_list:
    pos_string = get_sentence_pos_str(sentence,nlp)
    pos_ls = get_sentence_pos_list(sentence,nlp)
    pos_list.append(pos_ls)
    pos_strings.append(pos_string)
annotation['pos_string']=pos_strings
annotation['pos_list']=pos_list

annotation = annotation[annotation['to_exclude']==0]
annotation = annotation[['text','label','pos_string','pos_list']]

In [50]:
lstm_set = LSTMDataset()

a=lstm_set.tokenizer_class.fit_tokenizer_on_train(annotation['pos_string'].tolist())

In [53]:
type(annotation['pos_string'][0])

str

In [62]:
a[0] / (a[1]+a[0])

0.44329896907216493

In [58]:
a = annotation.value_counts('label')